# Load Data

In [1]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import mnet

Create path to access local files

In [2]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()

/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: local metadata cache: starting processing of paths: 2
/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1594150774.3113716
bundles.py: WARNING: Skipped 2 sources already in cache.
/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: writing time: 0.026 md items: 2

/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
caching.py: WARNING: cache: final writing time: 0.026 seconds


Save Telemann Fantasie No. 1 as a stream object

In [3]:
s = music21.corpus.parse('telemannfantasie1.xml')
chord_lst = s.flat.chordify().recurse().notes


In [4]:
offsets=[0.0, 16.0, 40.0, 104.0, 144.0, 162.0, 180.0, 201.0, "end"]


In [5]:

def convert_grouped_rn(chord_lst, offsets, key):
    
    nodelst=[]
    transition_lst=[]
    i=0
    g=1
    node_group=offsets[0]    
    
    while i < len(chord_lst):
        
        '''
        Extract note + rn
        '''
        chord=chord_lst[i]
        #extract melody
        mel = max(chord.pitches)
        #extract harmony
        harm = chord.remove(mel)
        rn = music21.roman.romanNumeralFromChord(chord, music21.key.Key(key))
        rn=str(rn).split()[1]
        
        '''
        determine group
        '''
        print("looking for ", offsets[g] )
        print("currently at ", chord.offset)
        if chord.offset == offsets[g]:
            node_group = offsets[g]
            g+=1
            if i !=0:
                transition_lst.append((nodelst[i-1],\
                     str(mel)+" "+rn+" "+str(node_group)))
                
        node = str(mel)+" "+rn+" "+str(node_group)
        nodelst.append(node)
        i +=1
        

    return nodelst, transition_lst


In [6]:
offsets=[0.0, 16.0, 40.0, 104.0, 144.0, 162.0, 180.0, 201.0, "end"]
nodelst_group, transition_edges=mnet.convert_grouped_rn(chord_lst, offsets, "A")

In [7]:
transition_edges

[('A4 I 0.0', 'D5 IV64 16.0'),
 ('E4 V6 16.0', 'A4 I 40.0'),
 ('A4 I 40.0', 'A5 I 104.0'),
 ('E4 V6 104.0', 'C#5 I 144.0'),
 ('E3 vii 144.0', 'C#5 I 162.0'),
 ('E3 vii 162.0', 'D5 IV64 180.0'),
 ('A4 I 180.0', 'D5 IV64 201.0')]

Create Graph

In [80]:
g_group=mnet.create_graph(nodelst_group)
#Write to .gexf
nx.write_gexf(g_group, "group_rn.gexf")

Generate Random Walk

In [81]:
randomwalk_group=mnet.generate_randomwalk(g_group)

In [82]:
randomwalk_group

['A4 I 0.0',
 'B4 I 0.0',
 'A4 I 0.0',
 'E4 I 0.0',
 'A4 I 0.0',
 'E4 I 0.0',
 'A4 I 0.0',
 'D5 IV64 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'E5 I 16.0',
 'C#5 I 16.0',
 'A4 I 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'A5 I 16.0',
 'E5 I 16.0',
 'C#5 I 16.0',
 'A5 I 16.0',
 'E5 I 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'A4 I 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'B4 I 16.0',
 'C#5 I 16.0',
 'A4 I 16.0',
 'E5 I 16.0',
 'C#5 I 16.0',
 'E5 I 16.0',
 'C#5 I 16.0',
 'A5 I 16.0',
 'C#5 I 16.0',
 'D4 IV64 16.0',
 'C#5 IV64 16.0',
 'B4 IV64 16.0',
 'A4 IV64 16.0',
 'D4 IV64 16.0',
 'C#5 IV64 16.0',
 'D5 IV64 16.0',
 'F#5 IV64 16.0',
 'D5 IV64 16.0',
 'F#5 IV64 16.0',
 'C#5 IV64 16.0',
 'B4 IV64 16.0',
 'A4 IV64 16.0',
 'D4 IV64 16.0',
 'C#5 IV64 16.0',
 'B4 IV64 16.0',
 'A4 IV64 16.0',
 'D#4 #ivob64 16.0',
 'C#5 #ivob64 16.0',
 'B4 #ivob64 16.0',
 'A4 #ivob64 16.0',
 'A5 #ivob64 16.0',
 'C

Convert random walk to back to music.

Uses conversion function group_strto16thnote, which takes a group pitch string and converts it to a note with length of 16th note. Notes preceeding transition edges will be half notes

In [83]:
def str_rn_group(randomwalk):
        
    notelist = []
    i=0
    randomwalk.append('pad long boo')
    #print(len(randomwalk))
    while i < len(randomwalk)-1:
        
        #print(i)
        string = randomwalk[i].split()
        #print(string)
        #note pitch
        notestr=string[0]
        n = music21.note.Note(notestr)
        group_cur = string[2]
        node_next = randomwalk[i+1]
        #print(node_next)
        group_next = node_next.split()[2]
        #print(group_cur, group_next)
        #note duration
        if group_cur !=group_next:
            n.duration.quarterLength =2
        else:
            n.duration.quarterLength = .25 
        
        
        notelist.append(n)
        i +=1
    return notelist  

In [84]:
tune = str_rn_group(randomwalk_group)
new_composition_group = mnet.convert_to_stream(tune)
#Write to MIDI
new_composition_group.write('xml', "group_rn.xml")

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.0 16.0
16.

104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 104.0
104.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.0 144.0
144.

'/Users/kaitlinpet/Desktop/Coursework/networks/project/notebooks/group_rn.xml'

## InfoMap

In [87]:
g_weighted = mnet.convert_to_weighted(g_group, fraction = False)

In [88]:
#convert to pajek
nx.write_pajek(g_weighted, "../graphs/group_rn.net")